In [5]:
include("../ode_solver.jl")
include("../numerical_shooting.jl")
include("../examples/example_functions.jl")
include("../visualisation.jl")
include("../finite_difference.jl")
include("../numerical_continuation.jl")

pseudo_arclength (generic function with 1 method)

In [18]:
using ForwardDiff
t = 1:0.1:100
a = [1 5 0]
ForwardDiff.jacobian((x) -> hopf3d(x, t), a)

3×3 Matrix{Float64}:
 -27.0  -11.0   0.0
  -9.0  -75.0   0.0
   0.0    0.0  -1.0

In [30]:
function np_continuation(f, x0, T, parameter, par_values; discretisation, arg...)
    
    """
    Attempts to find a function's solution for each parameter value using the last found solution as an initial guess.
    First solution is found using the inputted initial conditions, x0.

        Parameters:
            f (function): Function which returns a singular value or 1 x n matrix of values.
                The parameter 
            x0 (matrix): Matrix of initial values in the 1 x n form, eg: [1] or [1 1].
            T (float): Initial guess for the period.
            parameter: The parameter in the system to vary.
                Allowable method inputs: a, alpha, b, beta, c, d, delta, sigma
            par_values (array or range): Parameter values to solve between.
            discretisation (string): The discretisation to use, either "shooting" or "none."
            arg (list, optional): Arguments to pass to f.

        Returns:
            par_values, conditions: the parameter values and corresponding solutions.

        Example Usage:
            np_continuation(hopf2d, [1 1], 6, 0:0.01:2, discretisation="shooting")
    """

    x0 = [x0 T]
    x0 = [Float64(number) for number in x0] # convert ints to floats for use in nlsolve
    

    if discretisation == "shooting"

        if parameter == "a"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, a=par)
        elseif parameter == "alpha"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, alpha=par)
        elseif parameter == "b"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, b=par)
        elseif parameter == "beta"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, beta=par)
        elseif parameter == "c"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, c=par)
        elseif parameter == "d"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, d=par)
        elseif parameter == "delta"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, delta=par)
        elseif parameter == "sigma"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, sigma=par)
        end

    elseif discretisation == "none"
        
        if parameter == "a"
            discretisation = (u0, par) -> f(u0, a=par)
        elseif parameter == "alpha"
            discretisation = (u0, par) -> f(u0, alpha=par)
        elseif parameter == "b"
            discretisation = (u0, par) -> f(u0, b=par)
        elseif parameter == "beta"
            discretisation = (u0, par) -> f(u0, beta=par)
        elseif parameter == "c"
            discretisation = (u0, par) -> f(u0, c=par)
        elseif parameter == "d"
            discretisation = (u0, par) -> f(u0, d=par)
        elseif parameter == "delta"
            discretisation = (u0, par) -> f(u0, delta=par)
        elseif parameter == "sigma"
            discretisation = (u0, par) -> f(u0, sigma=par)
        end
       
    else
        error("Invalid discretisation.")
    end


    # Computation
    conditions = nlsolve((u) -> discretisation(u, par_values[1]), x0).zero
    for parameter in par_values[2:end]
        # solve using previous initial condition
        x = nlsolve((u) -> discretisation(u, parameter), conditions[[end],:]).zero
        conditions = [conditions; x]
    end

    return par_values, conditions
end

# par_values, conditions = np_continuation(algebraic, [1 2 5], 5, "c", -2:0.01:2, discretisation="none")
par_values, conditions = np_continuation(predprey, [1 1], 6, "b", 0.2:0.0005:0.3, discretisation="shooting")

(0.2:0.0005:0.3, [0.5778714846989212 0.2861488834424689 20.8168665836801; 0.5763691107296293 0.2865308479718483 20.78571055701764; … ; 0.2701562118655643 0.27015621186948413 17.93831071698523; 0.2701562118655643 0.27015621186948413 17.93831071698523])

In [31]:
# Create trace
u1 = scatter(
    x = par_values,
    y = conditions[:,1],
    mode="lines",
    name="u1",
    showlegend=true
    )
u2 = scatter(
    x = par_values,
    y = conditions[:,2],
    mode="lines",
    name="u2",
    showlegend=true
    )

layout = Layout(
    xaxis_title = "parameter",
    # yaxis_type="log",
    yaxis_exponentformat="power",
    # yaxis_title = "",
    width=700, height=350,
    )

plot([u1, u2], layout)

data: [
  "scatter with fields mode, name, showlegend, type, x, and y",
  "scatter with fields mode, name, showlegend, type, x, and y"
]

layout: "layout with fields height, margin, template, width, xaxis, and yaxis"

In [384]:
function algebraic(x; b=-1, c=0, arg...)

    return (x.^3 .+ x) .* c .+ b
end

algebraic (generic function with 3 methods)

In [385]:
function blahblah(u; arg...)

    x = algebraic(u; arg...)

    return x
end

blahblah (generic function with 2 methods)

In [388]:
x =  [1;2;1]
algebraic(x, b=[-1;-10;-100])


3-element Vector{Int64}:
   -1
  -10
 -100

In [389]:
elseif discretisation == "none"
        
    if parameter == "a"
        discretisation = (u0, par) -> f(u0, a=par)
    elseif parameter == "alpha"
        discretisation = (u0, par) -> f(u0, alpha=par)
    elseif parameter == "b"
        discretisation = (u0, par) -> f(u0, b=par)
    elseif parameter == "beta"
        discretisation = (u0, par) -> f(u0, beta=par)
    elseif parameter == "c"
        discretisation = (u0, par) -> f(u0, c=par)
    elseif parameter == "d"
        discretisation = (u0, par) -> f(u0, d=par)
    elseif parameter == "delta"
        discretisation = (u0, par) -> f(u0, delta=par)
    elseif parameter == "sigma"
        discretisation = (u0, par) -> f(u0, sigma=par)
    end
   
else
    error("Invalid discretisation.")
end

3-element Vector{Int64}:
   -1
  -10
 -100

In [322]:
getfield(Main, Symbol("beta"))

1